# Word embedding generation examples

## Using keras one_hot and pad_sequences functions

In [1]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [2]:
# define documents
docs = [
    'Well done!',
	'Good work',
	'Great effort',
    'nice work',
    'Excellent!',
    'Weak',
    'Poor effort!',
    'not good',
    'poor work',
    'Could have done better.'
]
# define class labels
labels = array([
    1,
    1,z
    1,
    1,
    1,
    0,
    0,
    0,
    0,
    0
])

In [3]:
# integer encode the documents
vocab_size = 50

# Index each work an return an arrays of document by with index's insted of words.
encoded_docs = [one_hot(d, vocab_size) for d in docs]
encoded_docs

[[11, 16],
 [10, 29],
 [20, 44],
 [12, 29],
 [46],
 [27],
 [40, 44],
 [22, 10],
 [40, 29],
 [39, 9, 16, 31]]

In [4]:
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[11 16  0  0]
 [10 29  0  0]
 [20 44  0  0]
 [12 29  0  0]
 [46  0  0  0]
 [27  0  0  0]
 [40 44  0  0]
 [22 10  0  0]
 [40 29  0  0]
 [39  9 16 31]]


In [5]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


## Using Pre-Trained GloVe Embedding 

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

import lib.embedding_utils as utils

In [8]:
# prepare tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(docs)

In [9]:
# integer encode the documents
encoded_docs = tokenizer.texts_to_sequences(docs)
print(encoded_docs)

[[6, 2], [3, 1], [7, 4], [8, 1], [9], [10], [5, 4], [11, 3], [5, 1], [12, 13, 2, 14]]


In [10]:
# integer encode the documents
encoded_docs = tokenizer.texts_to_sequences(docs)
print(encoded_docs)

[[6, 2], [3, 1], [7, 4], [8, 1], [9], [10], [5, 4], [11, 3], [5, 1], [12, 13, 2, 14]]


In [11]:
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[ 6  2  0  0]
 [ 3  1  0  0]
 [ 7  4  0  0]
 [ 8  1  0  0]
 [ 9  0  0  0]
 [10  0  0  0]
 [ 5  4  0  0]
 [11  3  0  0]
 [ 5  1  0  0]
 [12 13  2 14]]


Load the entire GloVe word embedding file into memory as a dictionary of word to embedding array.

In [12]:
tokenizer.word_index

{'work': 1,
 'done': 2,
 'good': 3,
 'effort': 4,
 'poor': 5,
 'well': 6,
 'great': 7,
 'nice': 8,
 'excellent': 9,
 'weak': 10,
 'not': 11,
 'could': 12,
 'have': 13,
 'better': 14}

In [13]:
embedding_matrix = utils.build_word_embedding(
    words=tokenizer.word_index.keys(),
    embedding_file='./word-embedding/glove.6B.100d.txt'
)

dict_keys(['have', 'not', 'could', 'well', 'work', 'good', 'great', 'better', 'done', 'effort', 'poor', 'weak', 'nice', 'excellent'])


In [14]:
vocab_size = embedding_matrix.shape[0]
embedding_vector_size = embedding_matrix.shape[1]

print("Word embedding shape: ", embedding_matrix.shape)
print("Vocabulary size: ", vocab_size)
print("Embedding vector size: ",embedding_vector_size)

Word embedding shape:  (15, 100)
Vocabulary size:  15
Embedding vector size:  100


Creare embedding matrix 

In [15]:
# Define model
model = Sequential()
model.add(Embedding(
    vocab_size, 
    embedding_vector_size, 
    weights=[embedding_matrix], 
    input_length=4, 
    trainable=False
))
model.add(Flatten())
model.add(Dense(
    units=1, 
    activation='sigmoid'
))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 4, 100)            1500      
_________________________________________________________________
flatten_2 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 401       
Total params: 1,901
Trainable params: 401
Non-trainable params: 1,500
_________________________________________________________________
None
